In [ ]:
import pickle
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 16})
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
import pandas as pd
from my_functions import z_NB
import numpy as np

In [ ]:
nbs_list = [[1, 5], [4, 8], [7, 11], [10, 14], [13, 17], [16, 20]]
# nbs_list = [[1, 20]]

df = None
for [nb_min, nb_max] in nbs_list:
    foldername = '/home/alberto/cosmos/LAEs/Luminosity_functions'
    selname = f'LF_r17-24_nb{nb_min}-{nb_max}_ew30_ewoth100_nb_1.0'
    filename = f'{foldername}/{selname}/selection.npy'
    with open(filename, 'rb') as f:
        selection = pickle.load(f)
    selection.keys()

    # Make DataFrame
    # data for dataframe:
    data_df = {}
    nice_nice = np.array(selection['nice_nice'])
    for key, value in selection.items():
        if key == 'puri':
            data_df[key] = value
        else:
            data_df[key] = np.array(value)[nice_nice]

    if df is None:
        df = pd.DataFrame(data=data_df)
    else:
        df = pd.concat([df, pd.DataFrame(data=data_df)])
# Drop duplicates
df = df.drop_duplicates(subset='RA')
df['tile_id'] = df['tile_id'].astype(int)
df.head()

In [ ]:
tile_dict = {
    2241: 'AEGIS001',
    2243: 'AEGIS002',
    2406: 'AEGIS003',
    2470: 'AEGIS004',
    2520: 'J-NEP',
    0: 'Total'
}
N_total = {
    'AEGIS001': 14292,
    'AEGIS002': 10025,
    'AEGIS003': 11576,
    'AEGIS004': 10584,
    'J-NEP': 17446,
    'Total': 63923
}

sel_data = []
for tile_id, tile_name in tile_dict.items():
    if tile_id == 0:
        this_mask = np.ones_like(df['src']).astype(bool)
    else:
        this_mask = (df['tile_id'] == tile_id)
    mask1 = df['pm_mask'] * this_mask
    mask2 = df['snr_mask'] * this_mask
    mask3 = df['ml_mask'] * this_mask
    mask4 = df['color_mask'] * this_mask
    nice_nice = df['nice_nice'] * this_mask
    sel_data.append(
        [tile_name,
         N_total[tile_name],
         f'{sum(this_mask)}({sum(this_mask) / N_total[tile_name] * 100:0.2f}%)',
         f'{sum(mask1)}({sum(mask1) / N_total[tile_name] * 100:0.2f}%)',
         f'{sum(mask2)}({sum(mask2) / N_total[tile_name] * 100:0.2f}%)',
         f'{sum(mask3)}({sum(mask3) / N_total[tile_name] * 100:0.2f}%)',
         f'{sum(mask4)}({sum(mask4) / N_total[tile_name] * 100:0.2f}%)',
         f'{sum(nice_nice)}({sum(nice_nice) / N_total[tile_name] * 100:0.2f}%)',
         ]
    )

col_names = ['Tile', 'Parent sample','3sigma, EW0>30 A', 'S/N > 6', 'Multiple line check',
             'proper motion', 'Color cut', 'Final selection']
sel_df = pd.DataFrame(data=sel_data, columns=col_names)
sel_df.to_csv('csv/sel_table.csv')
sel_df

In [ ]:
xim = df['x_im'].to_numpy().astype(float)
yim = df['y_im'].to_numpy().astype(float)

In [5]:
from load_jpas_catalogs import load_minijpas_jnep

pm_flx, pm_err, x_im, y_im, tile_id, number, starprob, spCl,\
        photoz, photoz_chi_best, photoz_odds, RA, DEC = load_minijpas_jnep(selection=True)

In [ ]:
pz = photoz[df['src']]
odds = photoz_odds[df['src']]
chi_best = photoz_chi_best[df['src']]

In [ ]:
# Save to npy/selection.npy
df.to_csv('csv/selection.csv')
df.shape

In [ ]:
# Drop bad NB image rows
# bad_NB_image = np.array([4380, 30395, 30513, 30977, 40306, 43721, 11771, 2583])
bad_NB_image = []

mask_bad_NB = np.ones_like(pz).astype(bool)
for bad_nb in bad_NB_image:
    wh = df['src'] == bad_nb
    mask_bad_NB[wh] = False

df = df[mask_bad_NB]

# Drop some columns
for col in ['src', 'x_im', 'y_im', 'other_lines', 'L_lya_NV']:
    df = df.drop(col, axis=1)

# Convert tile_id into tile names
tile_dict = {
    2241: 'AEGIS001',
    2243: 'AEGIS002',
    2406: 'AEGIS003',
    2470: 'AEGIS004',
    2520: 'J-NEP'
}

def tile_id_to_name(id):
    return tile_dict[int(id)]

def dec_to_sex(alpha):
    alpha = alpha * 60 * 60
    m, s = divmod(alpha, 60)
    d, m = divmod(m, 60)
    return f'{d:0.0f}º{m:0.0f}``{s:0.1f}\''

def redshift(nb):
    return f'{z_NB(nb)[0]:0.2f}'

def substitute_nans_zspec(what):
    if np.isnan(what):
        return '-'
    else:
        return f'{np.abs(what):0.2f}'

def substitute_nans_spcl(what):
    if what == 'nan':
        return '-'
    else:
        return what

def format_puri(p):
    return f'{p:0.2f}'

nb_Arr = np.array(df['nb_sel']).astype(int)

df['tile_id'] = df['tile_id'].astype(int).apply(tile_id_to_name)
df['RA'] = df['RA'].apply(dec_to_sex)
df['DEC'] = df['DEC'].apply(dec_to_sex)
df['nb_sel'] = df['nb_sel'].apply(redshift)
df['SDSS_spCl'] = df['SDSS_spCl'].astype(str).apply(substitute_nans_spcl)
df['SDSS_zspec'] = df['SDSS_zspec'].apply(substitute_nans_zspec)
df['puri'] = df['puri'].apply(format_puri)

ew_np = df['EW_lya'].to_numpy()
ew_err_np = df['EW_lya_err'].to_numpy()
ew_str = np.zeros(ew_np.shape).astype(str)
for i, (ew, err) in enumerate(zip(ew_np, ew_err_np)):
    ew_str[i] = f'{ew:0.0f}$\pm${err:0.0f}'
df['EW_lya'] = ew_str
df = df.drop('EW_lya_err', axis=1)

L_np = df['L_lya'].to_numpy()
L_err_np = df['L_lya_err'].to_numpy()
L_str = np.zeros(L_np.shape).astype(str)
for i, (L, err) in enumerate(zip(L_np, L_err_np)):
    L_str[i] = f'{L:0.2f}$\pm${err:0.2f}'
df['L_lya'] = L_str
df = df.drop('L_lya_err', axis=1)

mag = np.array(df['r'])
df = df.drop('r', axis=1)

# Sort by tile_id
df = df.sort_values(['tile_id', 'nb_sel'])
df.tail()

In [ ]:
sum(df['tile_id'] == 'AEGIS004')

In [ ]:
df.to_csv('csv/selection_table.csv')

In [ ]:
sum(df['SDSS_zspec'] != '-')

In [ ]:
L_lya = np.array([L_text[:5] for L_text in df['L_lya']]).astype(float)
zspec = np.array([0 if item == '-' else float(item) for item in df['SDSS_zspec']])
z_Arr = df['nb_sel'].to_numpy().astype(float)
EW_Arr = ew_np
puri = df['puri'].to_numpy().astype(float)
df.keys()

In [ ]:
xmsdss = pd.read_csv('csv/xmatch_sdss_dr12.csv', sep=',', header=1)
zph = xmsdss['zph'].to_numpy()
e_zph = xmsdss['e_zph'].to_numpy()

In [ ]:
from my_functions import count_true
sdss_mask = (df['SDSS_spCl'] != '-') 
puri_mask = (puri > 0.)
L_mask = (L_lya > 40)
cont_mask = sdss_mask & L_mask & puri_mask & (np.abs(zspec - z_Arr) > 0.3)
conf_mask = sdss_mask & L_mask & puri_mask & ~(np.abs(zspec - z_Arr) > 0.3)
N_cont = count_true(cont_mask)
N_confirmed = count_true(conf_mask)

perc = N_confirmed / (N_confirmed + N_cont)
perc_e = ((N_confirmed + N_cont)**-4 * (N_cont**2 * N_confirmed + N_confirmed**2 * N_cont))**0.5
print(f'Confirmed sources: {N_confirmed} / {(N_confirmed + N_cont)} ; p = {perc:0.2f}({perc_e:0.2f})')

avgpuri_total = np.mean(puri[L_mask & puri_mask])
avgpuri_sdss = np.mean(puri[sdss_mask & L_mask & puri_mask])
print(f'Average Total purity = {avgpuri_total:0.3f}')
print(f'Average SDSS purity = {avgpuri_sdss:0.3f}')

In [ ]:
from my_functions import central_wavelength, nb_fwhm


fig, ax = plt.subplots(figsize=(5, 2.5))

ax.plot(zspec[conf_mask], z_Arr[conf_mask],
        marker='o', ls='', c='green', markerfacecolor='none',
        markeredgewidth=2)
ax.plot(zspec[cont_mask], z_Arr[cont_mask],
        marker='o', ls='', c='red', markerfacecolor='none',
        markeredgewidth=2)

# lines
zsp_xx = np.linspace(0, 5, 50)
w_lya = 1215.67
w_CIV = 1549.48
w_CIII = 1908.73
w_MgII = 2799.12
ax.plot(zsp_xx, zsp_xx, ls='--', c='orange', label='Ly$\alpha$', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_CIV - 1, zsp_xx, 
        ls='--', c='k', label='CIV', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_CIII - 1, zsp_xx, 
        ls='--', c='k', label='CIII', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_MgII - 1, zsp_xx, 
        ls='--', c='k', label='MgII', zorder=-99)

text_plot = [[0.6, 3.5, 'MgII'],
             [1.55, 3.5, 'CIII'],
             [2.25, 3.5, 'CIV'],
             [3.2, 3.5, r'Ly$\alpha$']]
for text in text_plot:
        ax.text(text[0], text[1], text[2], fontsize=10)

# ax.set_yscale('log')
ax.set_ylim(2, 4.1)
ax.set_xlim(0, 4)
ax.set_xlabel('SDSS $z_\mathrm{spec}$', fontsize=15)
ax.set_ylabel('$z_\mathrm{NB}$', fontsize=15)
ax.tick_params(direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.savefig('figures/zspec_vs_z_Arr.pdf',
        bbox_inches='tight', facecolor='w', pad_inches=0)
plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

ax.hist(photoz[(photoz_odds > 0.0) & (photoz_chi_best < 10000)],
        histtype='step', label='All photoz')
ax.hist(pz[(odds > 0.75) & (chi_best < 100)], histtype='step',
        label='Selection')
ax.set_yscale('log')
ax.legend(fontsize=10)

plt.show()

fig, ax = plt.subplots(figsize=(5, 2.5))

this_mask = (z_Arr > 0) & (odds > 0.0) & (chi_best < 10000) & ~(cont_mask | conf_mask)
ax.plot(pz[this_mask], z_Arr[this_mask],
        marker='o', ls='', c='gray', markerfacecolor='none',
        markeredgewidth=2)
this_mask = (z_Arr > 0) & (odds > 0.0) & (chi_best < 10000) & cont_mask
ax.plot(pz[this_mask], z_Arr[this_mask],
        marker='o', ls='', c='r', markerfacecolor='none',
        markeredgewidth=2)
this_mask = (z_Arr > 0) & (odds > 0.0) & (chi_best < 10000) & conf_mask
ax.plot(pz[this_mask], z_Arr[this_mask],
        marker='o', ls='', c='green', markerfacecolor='none',
        markeredgewidth=2)

# lines
zsp_xx = np.linspace(0, 5, 50)
w_lya = 1215.67
w_OII = 3727.10
w_OIII = (5008.24 + 4960.30) * 0.5
w_Hbeta = 4862.68
w_Halpha = 6564.61
w_MgII = 2799.12
lines_dict = {
        'Lya': w_lya,
        'OII': w_OII,
        'OIII': w_OIII,
        'Hbeta': w_Hbeta,
        'Halpha': w_Halpha
}
ax.plot(zsp_xx, zsp_xx, ls='--', c='orange', label='Ly$\alpha$', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_OII - 1, zsp_xx, 
        ls='--', c='k', label='OII', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_OIII - 1, zsp_xx, 
        ls='--', c='k', label='OIII', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_Hbeta - 1, zsp_xx, 
        ls='--', c='k', label='Hbeta', zorder=-99)
# ax.plot((zsp_xx + 1) * w_lya / w_Halpha - 1, zsp_xx, 
#         ls='--', c='k', label='Halpha', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_MgII - 1, zsp_xx, 
        ls='--', c='k', label='MgII', zorder=-99)

text_plot = [[0.65, 3.9, 'OII'],
             [0.07, 3.9, 'OIII'],
             [0.28, 3.9, 'Hbeta'],
             [1.2, 3.9, 'MgII']]
for text in text_plot:
        ax.text(text[0], text[1], text[2], fontsize=10)

# ax.set_yscale('log')
ax.set_ylim(2, 4.1)
ax.set_xlim(-0.1, 1.6)
ax.set_xlabel('$z_\mathrm{phot}$', fontsize=16)
ax.set_ylabel('$z_\mathrm{NB}$', fontsize=16)
ax.tick_params(direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.savefig('figures/zphot_vs_z_Arr.pdf',
        bbox_inches='tight', facecolor='w', pad_inches=0)
plt.show()

fig, ax = plt.subplots(figsize=(6, 4))

bins = np.linspace(1200, 5000, 15)
h =  ax.hist(w_lya * (1 + z_Arr[this_mask]) / (1 + pz[this_mask]), bins)
for name, w_value in lines_dict.items():
        if w_value > 9500:
                continue
        ax.axvline(w_value, color='dimgray', linestyle=':')
        ax.text(w_value + 0.1, h[0].max(), name,
                color='k', fontsize=11, in_layout=True)

ax.set_xlabel('$\lambda$ ($\AA$)', fontsize=15)

plt.show()

fig, ax = plt.subplots(figsize=(5, 2.5))

w_central = central_wavelength()
nb_fwhm_Arr = np.array(nb_fwhm(np.arange(60)))
z_min = (w_central[nb_Arr] - nb_fwhm_Arr[nb_Arr] * 0.5) / w_lya - 1
z_max = (w_central[nb_Arr] + nb_fwhm_Arr[nb_Arr] * 0.5) / w_lya - 1
yerr = (z_max - z_min) * 0.5

ax.errorbar(zspec[conf_mask], (z_Arr - zspec)[conf_mask], yerr=yerr[conf_mask],
            fmt='o', markerfacecolor='w', markeredgecolor='green',
            capsize=4, ecolor='green')
ax.axhline(0, ls='--', c='k', zorder=-99)

ax.set_ylim(-0.25, 0.25)
ax.set_ylabel('$z_\mathrm{NB} - z_\mathrm{spec}$', fontsize=16)
ax.set_xlabel('SDSS $z_\mathrm{spec}$', fontsize=16)
ax.tick_params(direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.savefig('figures/delta_zspec.pdf',
        bbox_inches='tight', facecolor='w', pad_inches=0)
plt.show()

print(np.mean((z_Arr - zspec)[conf_mask & (zspec > 2.2)]))

In [ ]:
def do_this():
        fig, ax = plt.subplots(figsize=(6, 3.5))

        bins = np.linspace(1, 4, 20 + 1)
        bins_c = [bins[i: i + 2].sum() * 0.5 for i in range(len(bins) - 1)]
        bins_sep = bins_c[1] - bins_c[0]
        EW_mock_good_h = np.load(f'npy/Expected_EW_hist.npy')
        EW_mock_cont_h = np.load(f'npy/Cont_EW_hist.npy')
        EW_mock_full = EW_mock_cont_h + EW_mock_good_h
        norm = EW_mock_full.sum()
        EW_mock_full /= norm
        EW_mock_laes = EW_mock_good_h
        EW_mock_laes /= norm

        EW_this_h = np.histogram(np.log10(EW_Arr), bins=bins)[0].astype(float)
        EW_this_h /= EW_this_h.sum()

        ax.plot(bins_c + bins_sep * 0.5, EW_this_h, drawstyle='steps', color='dimgray',
                zorder=99, label='miniJPAS + J-NEP')

        ax.plot(bins_c, EW_mock_full, color='darkorange', ls='--',
                linewidth=2, label='Mock (full selection)')
        ax.plot(bins_c, EW_mock_laes, color='red', ls=':',
                linewidth=2, label='Mock (only LAEs)')

        ax.set_ylabel('Fraction of sources', fontsize=16)
        ax.set_xlabel(r'log EW$_0^{\mathrm{Ly}\alpha}$ ($\AA$)', fontsize=16)

        ax.tick_params(labelsize=14, direction='in', which='both')
        ax.yaxis.set_ticks_position('both')
        ax.xaxis.set_ticks_position('both')
        
        ax.set_xlim(1, 4)
        ax.set_ylim(0, 0.3)

        ax.legend(fontsize=14)

        fig.savefig(f'figures/EW_hist.pdf', bbox_inches='tight',
                facecolor='w', edgecolor='w')
        plt.show()

do_this()

In [ ]:
from load_jpas_catalogs import load_minijpas_jnep
pm_flx, pm_err, tile_id, pmra_sn, pmdec_sn, parallax_sn, starprob, _,\
        spCl, zsp, photoz, photoz_chi_best, photoz_odds, N_minijpas, x_im, y_im,\
        ra, dec =\
        load_minijpas_jnep((['minijpas', 'jnep']))

In [ ]:
# The selection
selection = pd.read_csv('csv/selection_first_kinda.csv')
selection_last = pd.read_csv('csv/selection.csv')

path_to_files = '/home/alberto/almacen/Selected_LAEs/First_visual_sel'

# ATT inspection
vi_att = pd.read_csv(f'{path_to_files}/inspeccion_AT.csv')
score_att = vi_att.to_numpy()[:, 1].astype(int)
# SGL inspection
vi_sgl = pd.read_csv(f'{path_to_files}/Sid_classification_noInfo_iter2.csv')
score_sgl = vi_sgl.to_numpy()[:, 1].astype(int)
# AFS inspection
vi_afs = pd.read_csv(f'{path_to_files}/class_afs.csv')
score_afs = vi_afs.to_numpy()[:, 1].astype(int)

files_list = list(vi_sgl.to_numpy()[:, 4])
src_list = [s.replace('J-NEP', 'caca') for s in files_list]
src_list = [s.replace('-', ' ') for s in src_list]
src_list = [s.replace('.', ' ') for s in src_list]
src_list = [s.split() for s in src_list]
src_list = np.array([int(s[2]) for s in src_list])

bad_NB_image = np.array([5, 19, 21, 34, 36, 38, 39, 52, 55, 139, 140, 171])
for sc in [score_att, score_sgl, score_afs]:
    sc[bad_NB_image] = 0

selection.head()